# **🧬 BayesOpt**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

## **📡 Lennard-Jones Potential**

In [2]:
def intermolecular_pair_potential(coordinates, epsilon=1.0, sigma=1.0):
    total_energy = 0.0
    n = len(coordinates)
    for i in range(n - 1):
        for j in range(i + 1, n):
            r = np.linalg.norm(coordinates[i] - coordinates[j])
            if r == 0:
                continue
            total_energy += 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)
    return total_energy

def lj_wrapper(x, epsilon=1.0, sigma=1.0):
    coords = x.reshape(-1, 3)
    return intermolecular_pair_potential(coords, epsilon, sigma)

## **🧪 Expected Improvement**

In [3]:
def expected_improvement(X, X_sample, Y_sample, gpr, xi=0.01):
    mu, sigma = gpr.predict(X, return_std=True)
    mu_sample_opt = np.min(Y_sample)

    with np.errstate(divide='warn'):
        imp = mu_sample_opt - mu - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
    return ei

## **🔍 Acquisition Optimizer**

In [4]:
def propose_location(acq_func, X_sample, Y_sample, gpr, bounds, n_restarts=25):
    dim = bounds.shape[0]
    min_val = float("inf")
    min_x = None

    def min_obj(X):
        return -acq_func(X.reshape(1, -1), X_sample, Y_sample, gpr)

    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1], size=dim)
        res = minimize(min_obj, x0=x0, bounds=bounds, method="L-BFGS-B")
        if res.fun < min_val:
            min_val = res.fun
            min_x = res.x
    return min_x.reshape(1, -1)

## **🤖 Bayesian Optimization Loop**

In [5]:
def bayesopt_lj(n_particles=4, n_iter=15):
    dim = n_particles * 3
    bounds = np.array([[0.0, 1.0]] * dim)

    X_sample = np.random.uniform(0, 1, size=(5, dim))
    Y_sample = np.array([[lj_wrapper(x)] for x in X_sample])

    kernel = Matern(nu=2.5)
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=1e-6)

    for i in range(n_iter):
        gpr.fit(X_sample, Y_sample.ravel())
        X_next = propose_location(expected_improvement, X_sample, Y_sample, gpr, bounds)
        Y_next = lj_wrapper(X_next.ravel())
        X_sample = np.vstack((X_sample, X_next))
        Y_sample = np.vstack((Y_sample, [[Y_next]]))
        print(f"Iteration {i+1}: Energy = {Y_next:.6f}")

    best_idx = np.argmin(Y_sample)
    best_x = X_sample[best_idx].reshape(-1, 3)
    best_energy = Y_sample[best_idx][0]
    print("\nBest Energy Found:", best_energy)
    print("Best Configuration:\n", best_x)
    return best_x, best_energy

## **🚀 Running**

In [6]:
best_coords, final_energy = bayesopt_lj(n_particles=4, n_iter=15)

Iteration 1: Energy = 2989.195511
Iteration 2: Energy = 140991334856.977020


c:\Users\rwidd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 3: Energy = 64618851.801851
Iteration 4: Energy = 2292779.556089


c:\Users\rwidd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\rwidd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 5: Energy = 122576.850587


...

Iteration 13: Energy = 61318.293540


c:\Users\rwidd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 14: Energy = 44373725.743090


c:\Users\rwidd\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


Iteration 15: Energy = 690059.942090

Best Energy Found: 1288.3546940832605
Best Configuration:
 [[0.75865016 0.00281375 0.45488861]
 [0.32277493 0.6625115  0.38969452]
 [0.79612663 0.95936689 0.67905616]
 [0.23326595 0.01543737 0.9202505 ]]
